#### 向ai询问了调用api的方法和rag的实现

In [ ]:
import sys
import os
import time
import platform
import json
from pathlib import Path
import numpy as np
import requests

DEEPSEEK_API_KEY = "sk-2b814ccebc6e4d90b76ef40d2dd36f83"
DEEPSEEK_API_URL = "https://api.deepseek.com/v1"

class DeepSeekAPI:
    def __init__(self, api_key=None, api_base=None, timeout=30):
        self.api_key = api_key or os.getenv("DEEPSEEK_API_KEY")
        self.api_base = api_base or "https://api.deepseek.com/v1"
        self.timeout = timeout
        
        if not self.api_key:
            raise ValueError("API密钥未设置")
        
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
    
    def generate_text(self, messages, model="deepseek-chat", max_tokens=1024, temperature=0.2,top_p=0.8):
        endpoint = f"{self.api_base}/chat/completions"
        payload = {
            "model": model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p
        }
        
        try:
            response = requests.post(
                endpoint,
                headers=self.headers,
                json=payload,
                timeout=self.timeout
            )
            response.raise_for_status()
            data = response.json()
            content = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"success": True, "content": content, "raw_data": data}
        except Exception as e:
            return {"success": False, "error": f"API调用失败: {str(e)}"}

### 普通的问题

In [ ]:


system_prompt = {
    "role": "system",
    "content": f"简单回答用户问题,每个问题限制在20字内"
}
user_prompt = {
    "role": "user",
    "content": "帮我介绍⼀下华中科技⼤学优秀的学⽣社团 Unique Studio。我昨天没有写作业, 帮我想⼀个借⼝⾯对⽼师。我在实习期被辞退了, 依照法律, 我能够拿到补偿吗？张三说李四在说谎，李四说王五在说谎，王五说张三和李四都在说谎。请问到底谁在说谎？"
}

api = DeepSeekAPI(api_key=DEEPSEEK_API_KEY)
response = api.generate_text(messages=[system_prompt, user_prompt])

if response["success"]:
    print("结果:", response["content"].strip())
else:
    print(f"错误: {response.get('error', '未知错误')}")
    

## 使用RAG拓展DeepSeekAPI调用

## RAG增强下的deepseek回答（需要科学上网访问huggingface

In [ ]:
import os
import json
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.prompts import PromptTemplate

# 配置环境变量
os.environ["DEEPSEEK_API_KEY"] = "sk-2b814ccebc6e4d90b76ef40d2dd36f83"

# DeepSeek API调用函数
def call_deepseek_api(prompt, model="deepseek-chat"):
    """调用DeepSeek API生成回答"""
    url = "https://api.deepseek.com/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.environ['DEEPSEEK_API_KEY']}"
    }
    
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2,
        "max_tokens": 1024
    }
    
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"调用DeepSeek API时出错: {e}")
        return None

# 初始化向量数据库
def init_vector_db(documents_dir="paper"):
    """初始化向量数据库，加载知识库文档"""
    # 加载文档
    loader = DirectoryLoader(documents_dir, glob="*.txt", loader_cls=TextLoader)
    documents = loader.load()
    
    # 分割文档
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        separators=["\n\n", "\n", "。", "，", " ", ""]
    )
    splits = text_splitter.split_documents(documents)
    
    # 创建嵌入模型
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    # 创建向量存储
    vector_db = FAISS.from_documents(splits, embeddings)
    
    return vector_db

# RAG增强的问答函数
def rag_qa(vector_db, query):
    """结合RAG的问答功能"""
    # 检索相关文档
    retriever = vector_db.as_retriever(search_kwargs={"k": 3})  # 获取最相关的3个文档片段
    relevant_docs = retriever.get_relevant_documents(query)
    
    # 构建带上下文的提示
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    # 定义提示模板
    prompt_template = """你是一个问答专家，请基于以下提供的上下文信息来回答用户的问题。
如果上下文信息不足以回答问题，请明确说明无法回答，不要编造信息。

上下文信息:
{context}

用户的问题:
{query}

请给出你的回答:
"""
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "query"]
    ).format(context=context, query=query)
    
    # 调用DeepSeek API获取回答
    answer = call_deepseek_api(prompt)
    
    # 整理来源信息
    sources = [{"source": doc.metadata["source"], "content": doc.page_content[:100] + "..."} 
               for doc in relevant_docs]
    
    return {
        "answer": answer,
        "sources": sources
    }

# 主函数
def main():
    # 初始化向量数据库（首次运行会加载文档并创建向量库，后续可直接加载）
    try:
        # 尝试加载已存在的向量库
        embeddings = HuggingFaceEmbeddings(model_name="https://mirrors.aliyun.com/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/all-MiniLM-L6-v2")
        vector_db = FAISS.load_local("faiss_index", embeddings)
        print("成功加载已存在的向量数据库")
    except:
        # 不存在则创建新的向量库
        print("正在初始化新的向量数据库...")
        vector_db = init_vector_db()
        # 保存向量库供下次使用
        vector_db.save_local("faiss_index")
        print("向量数据库初始化完成并保存")
    
    # 示例查询
    while True:
        # query = input("\n请输入你的问题（输入'q'退出）: ")
        # if query.lower() == 'q':
        #     break
        
        query = "在2020年和2021年，由于疫情影响，全球健康预期寿命减少了多少年？在2019年，全球有多⼤⽐例的⼈⼝，其⾃付的医疗费⽤超过了家庭预算的10%？"
        result = rag_qa(vector_db, query)
        
        print("\n回答:")
        print(result["answer"])
        
        print("\n参考来源:")
        for i, source in enumerate(result["sources"], 1):
            print(f"{i}. 来源: {source['source']}")
            print(f"   内容片段: {source['content']}\n")

if __name__ == "__main__":
    main()


![RAG增强下的deepseek回答](ans.png)


## 普通调用下的deepseek回答

In [ ]:
system_prompt = {
    "role": "system",
    "content": f"回答用户问题"
}
user_prompt = {
    "role": "user",
    "content": "在2020年和2021年，由于疫情影响，全球健康预期寿命减少了多少年？在2019年，全球有多⼤⽐例的⼈⼝，其⾃付的医疗费⽤超过了家庭预算的10%？"
}

api = DeepSeekAPI(api_key=DEEPSEEK_API_KEY)
response = api.generate_text(messages=[system_prompt, user_prompt])

if response["success"]:
    print("结果:", response["content"].strip())
else:
    print(f"错误: {response.get('error', '未知错误')}")

![普通](ans_1.png)
